In [4]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import csv
import statsmodels.api as sm
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.preprocessing import LabelEncoder


In [5]:
df = pd.read_csv('train.csv')

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df = df.drop(columns=['sub_area'], axis=1)

In [8]:
X = df.loc[:, df.columns != 'price_doc']
y = df[['price_doc']]

In [9]:
object_cols = X.select_dtypes(include=['object']).columns

In [10]:
label = LabelEncoder()
for col in object_cols:
    X[col] = label.fit_transform(X[col].astype(str))

C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11660\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11660\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11660\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [11]:
X.shape

(178661, 270)

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [13]:
from sklearn.decomposition import PCA

PCA_s = PCA(n_components=30)
X = PCA_s.fit_transform(X)

In [14]:
from sklearn.feature_selection import VarianceThreshold

var = VarianceThreshold(threshold=0.01)
X = var.fit_transform(X)

In [15]:
skb = SelectKBest(f_regression, k=15)
X = skb.fit_transform(X, y)

c:\Users\Irtiza\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(3,include_bias=False)
X = poly.fit_transform(X)

In [17]:
df_test = pd.read_csv('test.csv')

In [18]:
df_test = df_test.drop(['row ID'], axis=1)

In [19]:
df_test = df_test.drop(['sub_area'], axis=1)

In [20]:
label_test = LabelEncoder()
for col in object_cols:
    df_test[col] = label_test.fit_transform(df_test[col].astype(str))

In [21]:
df_test = scaler.transform(df_test)

In [22]:
df_test = PCA_s.transform(df_test)

In [23]:
df_test = var.transform(df_test)

In [24]:
df_test = skb.transform(df_test)

In [25]:
df_test = poly.transform(df_test)

In [26]:
df_test.shape

(77789, 815)

In [28]:
# Creating a Linear Regression model
reg = LinearRegression()

# Fitting the model to the training data
reg.fit(X, y)

# Making predictions on the test data
y_pred_test = reg.predict(df_test)

# Save predictions to a CSV file
filepath = 'prediction_25nov_linearpoly.csv'
with open(filepath, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Add column headers
    writer.writerow(['row ID', 'price_doc'])
    
    # Writing predictions with corresponding index (starting from 1)
    for c, prediction in enumerate(y_pred_test, start=1):
        writer.writerow([c, prediction[0]])